In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

DIAGS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I080",
    "I081",
    "I340",
    "I350",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J398",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J90X",
    "J91X",
    "J931",
    "J955",
    "J980",
    "M348",
    "Q211",
    "Q231",
    "Q676",
    "T820",
    "T821",
    "Z450",
]

DIAGS_FALTANTES_PULMON = ["C349", "C348"]

DIAGS_RELEVANTES += DIAGS_FALTANTES_PULMON

In [3]:
# Carga los egresos nacionales
egresos_nacionales = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv", sep=";", encoding="latin-1"
)

# Filtra solamente los datos del INT
egresos_torax = egresos_nacionales.query("ESTABLECIMIENTO_SALUD == 112103").copy()

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [4]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_nacionales = (
    egresos_nacionales[egresos_nacionales["DIAG1"].isin(DIAGS_RELEVANTES)]
    .query("ANO_EGRESO >= 2017")
    .copy()
)

# Agrupa todos los diagnosticos relacionados a cancer de pulmon a un diagnostico comun
egresos_mas_relevantes_nacionales["DIAG1"] = egresos_mas_relevantes_nacionales["DIAG1"].replace(
    ["C340", "C341", "C342", "C343", "C348", "C349"], "C34"
)

# Filtra solamente los diagnosticos relevantes para el torax
egresos_mas_relevantes_torax = egresos_mas_relevantes_nacionales.query(
    "ESTABLECIMIENTO_SALUD == 112103"
).copy()

In [22]:
# Obtiene el resumen de metricas para el estrato nacional
metricas_diags_relevantes_nacionales = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_nacionales
)

# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_torax = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_torax
)

# Obtiene el porcentaje de resolucion de egresosx del Torax
porcentaje_egresos_nacionales_torax = (
    metricas_diags_relevantes_torax["n_egresos"] / metricas_diags_relevantes_nacionales["n_egresos"]
)

Una vez ya obtenido las metricas relevantes tanto para el estrato Nacional y del hospital a analizar, se procederá a comparar con los casos teóricos.


In [23]:
# Importa los casos teoricos por cada problema de salud
casos_teoricos = pd.read_excel("../data/interim/casos_teoricos_diagnosticos.xlsx")
casos_teoricos["Diagnostico"] = casos_teoricos["Diagnostico"].str.split("-").str[0].str.strip()
casos_teoricos = casos_teoricos.set_index("Diagnostico")

# Renombra columnas
casos_teoricos = casos_teoricos.rename(
    columns={
        "Poblacion 2017": 2017,
        "Poblacion 2018": 2018,
        "Poblacion 2019": 2019,
        "Poblacion 2020": 2020,
        "Poblacion 2021": 2021,
        "Poblacion 2022": 2022,
        "Poblacion 2023": 2023,
        "Poblacion 2024": 2024,
        "Poblacion 2025": 2025,
        "Poblacion 2026": 2026,
        "Poblacion 2027": 2027,
        "Poblacion 2028": 2028,
        "Poblacion 2029": 2029,
        "Poblacion 2030": 2030,
        "Poblacion 2031": 2031,
        "Poblacion 2032": 2032,
        "Poblacion 2033": 2033,
        "Poblacion 2034": 2034,
        "Poblacion 2035": 2035,
    }
)

# Obtiene los casos en el estrato nacional
casos_nacionales = casos_teoricos.query("Estrato == 'Pais' and Estadística == 'Incidencia'").copy()
casos_nacionales_del_2017_al_2020 = casos_nacionales[[i for i in range(2017, 2021)]]

Solamente se compararán los casos reales vs teóricos entre 2017 y 2020, ya que la información se encuentra disponible en tales periodos.

In [25]:
# Obtiene cuanto se hace cargo el INT de los casos teoricos
porcentaje_atencion_teorica_int = (
    metricas_diags_relevantes_torax["n_pacientes_distintos"].sum(axis=1)
    .div(casos_nacionales_del_2017_al_2020.sum(axis=1), axis=0)
)

In [54]:
# Se obtiene la cantidad de casos que debiese atender el Torax, asumiendo que atendera
# lo mismo que entre 2017 y 2020 del total de casos teoricos
cantidad_de_casos_estimados_int = casos_nacionales[[i for i in range(2017, 2036)]].mul(
    porcentaje_atencion_teorica_int, axis=0
)

# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_int = cantidad_de_casos_estimados_int.mul(
    metricas_diags_relevantes_torax["egresos_por_paciente_agrupado"]["2017-2020"], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_int = egresos_estimados_int.mul(
    metricas_diags_relevantes_torax["dias_estada_promedio_agrupado"]["2017-2020"], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_int = (dias_estada_estimados_int / 365.25) * 1.2

In [55]:
camas_estimadas_int.sum()

2017    46.189548
2018    47.212899
2019    48.343425
2020    49.501767
2021    50.408278
2022    51.173073
2023    51.905465
2024    52.628666
2025    53.345258
2026    54.054643
2027    54.757204
2028    55.451238
2029    56.132028
2030    56.793593
2031    57.451960
2032    58.080672
2033    58.694638
2034    59.297433
2035    59.887932
dtype: float64